In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/kaggle/input/next-word-prediction/1661-0.txt',sep='\t',header=None,names=["Texts"])

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Texts'])

In [ ]:
voc_size = len(tokenizer.index_word)
voc_size

In [ ]:
tokenized_sentence = tokenizer.texts_to_sequences(df['Texts'])
tokenized_sentence[0]

In [ ]:
input_sequence = []

for j in range(0,len(tokenized_sentence)):  
    for i in range(1,len(tokenized_sentence[j])):
        input_sequence.append(tokenized_sentence[j][:i+1])

In [ ]:
max_len = max([len(x) for x in input_sequence])
max_len

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequence = pad_sequences(input_sequence, maxlen = max_len, padding='pre')

In [ ]:
x= padded_input_sequence[:,:-1]
y = padded_input_sequence[:,-1]

In [ ]:
x[0]

In [ ]:
input_shape = len(x[0])
input_shape

In [ ]:
len(y)

In [ ]:
y = to_categorical(y,num_classes=voc_size+1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Embedding

model = Sequential()
model.add(Embedding(voc_size+1, 100, input_shape=(input_shape,)))
model.add(LSTM(150))
model.add(Dense(voc_size+1, activation='softmax'))

model.summary()

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(x,y,
    epochs=50,
    batch_size = 32
)

In [ ]:
import pickle
filename = 'nextWordModel.pkl'
pickle.dump(model, open(filename, 'wb'))

# Save the Totenizer

In [ ]:
pickle.dump(tokenizer, open("tokenizer.pkl", "wb"))

In [ ]:
loaded_model = pickle.load(open("/kaggle/working/nextWordModel.pkl", "rb"))
loaded_tokenizer = pickle.load(open("/kaggle/working/tokenizer.pkl", "rb"))


In [ ]:
# loaded_tokenizer

In [ ]:
text = "kill"

for  i in range(18):
    token_text = loaded_tokenizer.texts_to_sequences([text])[0]
    padded_token_text = pad_sequences([token_text],maxlen=19,padding='pre')
    pos = np.argmax(loaded_model.predict(padded_token_text))
    for word,index in loaded_tokenizer.word_index.items():
        if index == pos:
                text = text + " " + word
                print(text)